In [ ]:
######################################################
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import os, argparse
import cv2, spacy, numpy as np
from keras.models import model_from_json
from keras.optimizers import SGD
from sklearn.externals import joblib
from keras import backend as K
from keras.utils.vis_utils import plot_model
word_embeddings = spacy.load('en_vectors_web_lg')



In [ ]:
from keras.models import Model  
from keras.layers import *
from keras.applications.resnet50 import ResNet50  
from keras.preprocessing import image
import numpy as np

from keras.applications.inception_resnet_v2 import InceptionResNetV2 
from keras.applications.xception import Xception


model_resnet50 = ResNet50(include_top=True)
#print(model_resnet50.summary())
model_resnet50 = Model(inputs=model_resnet50.input, outputs=model_resnet50.get_layer('avg_pool').output)



In [ ]:
######################################################


def get_image_features_resnet(x):
    x = keras.applications.resnet50.preprocess_input(x)
    features = model_resnet50.predict(x)
    return features

In [ ]:
######################################################

def get_image_features_inception(x):
    x = keras.applications.inception_resnet_v2.preprocess_input(x)
    features = model_inception.predict(x)
    return features

In [ ]:
######################################################


def get_image_features_xception(x):
    x = keras.applications.xception.preprocess_input(x)
    features = model_xception.predict(x)
    return features

In [ ]:
def get_question_features(question):
    
    

    tokens = word_embeddings(question)
    question_tensor = np.zeros((1, 30, 300))
    for j in range(len(tokens)):
        question_tensor[0,j,:] = tokens[j].vector
    return question_tensor

In [ ]:
def pre_pro(image_file_name):
    img_path = image_file_name
    img = image.load_img(img_path, target_size=(299, 299))
    img2 = image.load_img(img_path, target_size=(224, 224))

    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x1 = image.img_to_array(img2)
    x1 = np.expand_dims(x1, axis=0)
    r = get_image_features_resnet(x1)
    i = get_image_features_inception(x)
    x = get_image_features_xception(x)
    return r,i,x

In [ ]:
######################################################

import json

   
train = []
test = []
imdir= 'train2014/COCO_train2014_%012d.jpg'


#print 'Loading annotations and questions...'
train_anno = json.load(open('v2_mscoco_train2014_annotations.json', 'r'))


train_ques = json.load(open('v2_OpenEnded_mscoco_train2014_questions.json', 'r'))

test_ques = json.load(open('v2_OpenEnded_mscoco_test2015_questions.json', 'r'))

#subtype = 'train2014'
for i in range(len(train_anno['annotations'])):
    ans = train_anno['annotations'][i]['multiple_choice_answer']
    question_id = train_anno['annotations'][i]['question_id']
    image_path = imdir%(train_anno['annotations'][i]['image_id'])
    img_id = train_anno['annotations'][i]['image_id']
    question = train_ques['questions'][i]['question']

    train.append({'ques_id': question_id,'img_id':img_id , 'img_path': image_path, 'question': question, 'ans': ans})





#json.dump(train, open('vqa_raw_train.json', 'w'))
#json.dump(test, open('vqa_raw_test.json', 'w'))



   
    

In [ ]:
######################################################

train = sorted(train, key = lambda i: (i['ques_id']))
#train = np.asarray(train)
#train = np.squeeze(train)
#np.save('train.npy',train)
train = train[0:80000]
print(len(train))

In [ ]:
######################################################
from sklearn.externals import joblib

label_encoder_file_name = 'FULL_labelencoder_trainval.pkl'
labelencoder = joblib.load(label_encoder_file_name)


#labelencoder.classes_[12]
mapp = {}
mappr = {}

for i in range(len(labelencoder.classes_)):
  #print(labelencoder.classes_[i],i)
  mapp[i] = labelencoder.classes_[i]
  mappr[labelencoder.classes_[i]] = i
  
#mapp[64]
#mappr[]


In [ ]:
######################################################

import time
import keras



imdir= 'train2014/COCO_train2014_%012d.jpg'
#word_embeddings = spacy.load('en_vectors_web_lg')




start = time.time()
imgfet_res = []
imgfet_inc = []
imgfet_x = []
qfet = []



for i in range(80000):
    try:
        mappr[train[i]['ans']]
        fet_res,fet_inc,fet_x = pre_pro(imdir%(train[i]['img_id']))
        print('img')
        u = str(train[i]['question'])
        qfet = get_question_features(u)
        print('ques')
        imgfet_res.append(fet_res)
        imgfet_inc.append(fet_inc)
        imgfet_x.append(fet_x)

        quesfet.append(qfet)
        l.append(mappr[train[i]['ans']])
        print(i)
        end = time.time()
        g = (end-start)
        #et = tt-g
        #print('estimated time : ',end)
        print(time.strftime("%H:%M:%S", time.gmtime(g)))

        
        
        
        
        
        
        
    except KeyError:
        continue

In [ ]:
import numpy as np
#np.save('imgfet_res.npy',imgfet_res)

imgfet_res = np.asarray(imgfet_res)
imgfet_res = np.squeeze(imgfet_res)
np.save('imgfet_res.npy',imgfet_res)
del imgfet_res


In [ ]:
imgfet_inc = np.asarray(imgfet_inc)
imgfet_inc = np.squeeze(imgfet_inc)
np.save('imgfet_inc.npy',imgfet_inc)

del imgfet_inc

In [ ]:
imgfet_x = np.asarray(imgfet_x)
imgfet_x = np.squeeze(imgfet_x)
np.save('imgfet_x.npy',imgfet_x)

del imgfet_x


In [ ]:
qfet = np.asarray(qfet)
qfet = np.squeeze(qfet)
np.save('quesfet.npy',qfet)


In [ ]:
l = []
for i in range(80000):
    
    
    
  #fet = get_image_features(imdir%(train[i]['img_id']))
  try:
    mappr[train[i]['ans']]
    l.append(mappr[train[i]['ans']])
    print(train[i]['ans'])
    
  except KeyError:
    continue
    
    
print(len(l))

  

In [ ]:
l = np.asarray(l)
l = np.squeeze(l)

np.save('label.npy',l)
